In [1]:
import requests 
import hashlib
from collections import defaultdict
import json
import os
from openai import OpenAI
from tqdm.auto import tqdm
import pickle
import pandas as pd

/Users/viviensiew/.local/share/virtualenvs/llm-zoomcamp-0DI9tvdQ/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

# documents = []

# for course in documents_raw:
#     course_name = course['course']

#     for doc in course['documents']:
#         doc['course'] = course_name
#         documents.append(doc)

In [3]:
documents_raw = []
doc_json = "../pre-course workshop 1/documents.json"

if not os.path.isfile(doc_json):
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
else:
    print(f"File already exists at {doc_json}.")
    with open(doc_json, 'rt') as f_in:
        documents_raw = json.load(f_in)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

File already exists at ../pre-course workshop 1/documents.json.


### Generate unique ID for each document
To relate back the generated question to the original document record.

An option is to generate using running sequence however this could pose an issue if someone updates the FAQ and broke the original sequence. Hence an alternative is used for each record:
* Extract texts from the fields.
* Generate an MD5 hash using the extracted texts that are encoded in UTF-8.
* Convert the MD5 hash into hexadecimal digits, then take only the first 8 chars.

Pros: Independent of sequence.

Cons: Dependent on record text. Breaks if text is changed.

In [4]:
def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [6]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

### Generate a dictionary of ids
Allows comparison between number of document records and records that share the same unique ID.

Each ID can contain one or more document records in its value list.

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
len(hashes), len(documents)

(947, 948)

Examine the hash with more than one document records.

In [9]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [10]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

### Save the documents.json with unique IDs

In [11]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


### LLM to generate the 5 questions per record
Note that Ollama is used here instead of GPT-4o.

Encourage LLM to use fewer words to generate the questions so as to test how well the retrieval system works.

In [13]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [14]:
# client = OpenAI()
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
# def generate_questions(doc):
#     prompt = prompt_template.format(**doc)

#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )

#     json_response = response.choices[0].message.content
#     return json_response
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [16]:
results = {}

### Test the question generation 
Test with one record

In [17]:
doc = documents[2]
prompt = prompt_template.format(**doc)
print(prompt)

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [18]:
response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": prompt}]
)

json_response = response.choices[0].message.content
json_response

' ["Can I enroll in the course after its commencement if I haven\'t registered?", "Is there a possibility to join post-deadline for homework submissions but still complete final project on time?", "How are late entries treated in terms of course participation and deadlines?", "Are last-minute registrations considered, especially concerning assignment turn-ins?", "What is the consequence of not registering by the start date, regarding assignment submission?"]'

### Generate questions
* For each record, generate 5 questions.
* Note that this could cause a rate limit error during the process and this code block will have to rerun, hence there's an error handling "if doc_id in results" to skip over records that are already processed, <b>skip over will not work if kernel is restarted</b>.
* Costs around $4 if run using GPT-4o.
* Took over 7H 17m to run using Ollama Phi3 in CPU

In [18]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|██████████████████████████████████████████████████████████████████████| 948/948 [7:17:04<00:00, 27.66s/it]


In [20]:
results

{'c02e79ef': ' ["When is the course scheduled to begin?", "Could you confirm the start date and time of the course?", "How can I join the live session at the beginning of the course?", "Where should I register for this course?", "Are there any additional channels or platforms where updates will be shared?"]',
 '1f6520ca': ' ["What specific prerequisites are required to enroll in the GitHub - DataTalksClub data-engineering-zoomcamp course?", "Could you detail the prerequisites listed under data-engineering-zoomcamp#prerequisites for this course?", "Are there any foundational skills or knowledge needed before taking GitHub - DataTalksClub\'s data-engineering-zoomcamp, as indicated in the prerequisites?", "What are the requirements mentioned under \'data-engineering-zoomcamp#prerequisites\' for participating in this course?", "Can you explain the necessary criteria detailed in the prerequisites for GitHub - DataTalksClub data-engineering-zoomcamp to gain admission into the course?"]',
 '7

### Save to binary file
The results dictionary consists of JSON format hence saved as binary file.

For the ground-truth question set generated by Ollama Phi3 model, it's saved as results-ollama.bin.

However, the subsequent code blocks will be run using the "results.bin" provided by this course.

In [19]:
# results_file = "results-ollama.bin"
# with open(results_file, 'wb') as f_out:
#     pickle.dump(results, f_out)
#     print(f'Results saved successfully to file {results_file}')

In [20]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [21]:
results

{'c02e79ef': '["When does the course begin?", "How can I get the course schedule?", "What is the link for course registration?", "How can I receive course announcements?", "Where do I join the Slack channel?"]',
 '1f6520ca': '["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]',
 '7842b56a': '[\n  "Can I enroll in the course after it starts?",\n  "Is late registration possible?",\n  "Am I eligible to submit homework if I join late?",\n  "Are there deadlines for final projects if I join late?",\n  "Can I submit all assignments at the end of the course if I start late?"\n]',
 '0bbf41ec': '[\n  "When will I receive the confirmation email after registering for the Data Engineering Bootcamp?",\n  "Do I need the confirmation email to start the Data Engineering Bootcamp?",\n  "Can I be

In [22]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

### Parse the result
Parse the generated questions in JSON format. 

Note that there could be invalid JSON formatting from the results which will require manual intervention, e.g. data cleanup in results by:
* Get the results key-item pair with invalid format.
* Change it manually in code and pass it back to results using dict key.

In [28]:
parsed_result = {}

for doc_id, json_questions in results.items():
    parsed_result[doc_id] = json.loads(json_questions)

### Example of invalid JSON Formatting and intervention

In [24]:
# print(len(parsed_result))
# print(doc_id)
# print(json_questions)

65
58c9f99f
[
"How can I resolve the Docker error 'invalid mode: \\Program Files\\Git\\var\\lib\\postgresql\\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for \\\Program Files\\Git\\var\\lib\\postgresql\\data'?"
]


In [27]:
# json_questions = [
# r"How can I resolve the Docker error 'invalid mode: \\Program Files\\Git\\var\\lib\\postgresql\\data'?",
# "What should I do if I encounter an invalid mode error in Docker on Windows?",
# "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
# "Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
# r"How do I correct the mounting path error in Docker for \\\Program Files\\Git\\var\\lib\\postgresql\\data'?"
# ]

# results[doc_id] = json.dumps(json_questions)

### Save the generated questions

Save in CSV: Question, course and document ID.

Examine the document ID index

In [29]:
doc_index = {d['id']: d for d in documents}

In [30]:
final_results = []

for doc_id, questions in parsed_result.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [31]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])
df.to_csv('ground-truth-data.csv', index=False)

In [32]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca


### Note
The csv file generated using results-ollama.bin can be found in ground-truth-data-ollama.csv